# సెమాంటిక్ కర్నెల్

ఈ కోడ్ నమూనాలో, మీరు [సెమాంటిక్ కర్నెల్](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ఫ్రేమ్‌వర్క్‌ను ఉపయోగించి ఒక ప్రాథమిక ఏజెంట్‌ను సృష్టిస్తారు.

ఈ నమూనా యొక్క లక్ష్యం, వివిధ ఏజెంటిక్ నమూనాలను అమలు చేసే అదనపు కోడ్ నమూనాల్లో మేము ఉపయోగించే దశలను మీకు చూపించడం.


## అవసరమైన పైథాన్ ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
import json
import os

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## క్లయింట్ సృష్టించడం

ఈ ఉదాహరణలో, LLM యాక్సెస్ కోసం [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ను ఉపయోగిస్తాము.

`ai_model_id` ను `gpt-4o-mini` గా నిర్వచించారు. GitHub Models మార్కెట్‌ప్లేస్‌లో అందుబాటులో ఉన్న మరో మోడల్‌కు మోడల్‌ను మార్చి వివిధ ఫలితాలను చూడండి.

GitHub Models కోసం `base_url` ఉపయోగించే `Azure Inference SDK` ను ఉపయోగించడానికి, Semantic Kernel లోని `OpenAIChatCompletion` కనెక్టర్‌ను ఉపయోగిస్తాము. ఇతర మోడల్ ప్రొవైడర్ల కోసం Semantic Kernel ఉపయోగించడానికి [అందుబాటులో ఉన్న ఇతర కనెక్టర్లు](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) కూడా ఉన్నాయి.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ఏజెంట్‌ను సృష్టించడం

కింద, మేము `TravelAgent` అనే ఏజెంట్‌ను సృష్టిస్తున్నాము.

ఈ ఉదాహరణలో, మేము చాలా సులభమైన సూచనలను ఉపయోగిస్తున్నాము. మీరు ఈ సూచనలను మార్చి ఏజెంట్ ఎలా భిన్నంగా స్పందిస్తుందో చూడవచ్చు.


In [ ]:
AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions=AGENT_INSTRUCTIONS,
)

## ఏజెంట్లను నడపడం

ఇప్పుడు మేము `ChatHistory` ను నిర్వచించి, దానికి `system_message` ను జోడించడం ద్వారా ఏజెంట్‌ను నడపవచ్చు. మేము ఇంతకు ముందు నిర్వచించిన `AGENT_INSTRUCTIONS` ను ఉపయోగిస్తాము.

ఇవి నిర్వచించిన తర్వాత, మేము `user_inputs` ను సృష్టిస్తాము, ఇది వినియోగదారు ఏజెంట్‌కు పంపే సందేశం అవుతుంది. ఈ సందర్భంలో, మేము ఈ సందేశాన్ని `Plan me a sunny vacation` గా సెట్ చేసాము.

ఏజెంట్ ఎలా భిన్నంగా స్పందిస్తుందో చూడటానికి ఈ సందేశాన్ని మార్చడానికి సంకోచించకండి.


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    # First, let the agent introduce itself
    agent_name = None
    full_response: list[str] = []
    function_calls: list[str] = []

    # Buffer to reconstruct streaming function call
    current_function_name = None
    argument_buffer = ""

    # Invoke agent with a simple greeting to trigger introduction
    async for response in agent.invoke_stream(
        messages="Hello",
        thread=thread,
    ):
        thread = response.thread
        agent_name = response.name
        content_items = list(response.items)

        for item in content_items:
            if isinstance(item, FunctionCallContent):
                if item.function_name:
                    current_function_name = item.function_name

                # Accumulate arguments (streamed in chunks)
                if isinstance(item.arguments, str):
                    argument_buffer += item.arguments
            elif isinstance(item, FunctionResultContent):
                # Finalize any pending function call before showing result
                if current_function_name:
                    formatted_args = argument_buffer.strip()
                    try:
                        parsed_args = json.loads(formatted_args)
                        formatted_args = json.dumps(parsed_args)
                    except Exception:
                        pass  # leave as raw string

                    function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                    current_function_name = None
                    argument_buffer = ""

                function_calls.append(f"\nFunction Result:\n\n{item.result}")
            elif isinstance(item, StreamingTextContent) and item.text:
                full_response.append(item.text)

    # Display agent introduction
    html_output = ""
    if function_calls:
        html_output += (
            "<div style='margin-bottom:10px'>"
            "<details>"
            "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
            "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
            "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
            f"{chr(10).join(function_calls)}"
            "</div></details></div>"
        )

    html_output += (
        "<div style='margin-bottom:20px'>"
        f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
    )

    display(HTML(html_output))

    # Now process user inputs
    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దయచేసి, మూల భాషలో ఉన్న అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. ముఖ్యమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
